**1) Environment & GPU check**

In [1]:
!nvidia-smi -L || echo "No GPU?"
import torch, platform, sys, os
print("CUDA available:", torch.cuda.is_available())
print("Torch:", torch.__version__, "| Python:", sys.version.split()[0], "| OS:", platform.platform())

# Clean workspace path
%cd /content
!rm -rf /content/live-gts
!mkdir -p /content/live-gts && cd /content/live-gts


GPU 0: Tesla T4 (UUID: GPU-754ad22d-75dd-3e41-efbe-46d3db764664)
CUDA available: True
Torch: 2.8.0+cu126 | Python: 3.12.11 | OS: Linux-6.1.123+-x86_64-with-glibc2.35
/content


**2) Install dependencies**

In [2]:
%pip -q install ultralytics supervision opencv-python PyYAML

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 19.3 MB/s eta 0:00:00


**3) Mount Google Drive to save outputs**[link text](https://)

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
SAVE_DIR = "/content/drive/MyDrive/gts_artifacts"  # change if you like
os.makedirs(SAVE_DIR, exist_ok=True)
print("Saving artifacts to:", SAVE_DIR)


Mounted at /content/drive
Saving artifacts to: /content/drive/MyDrive/gts_artifacts


**4) Install Roboflow for dataset**

In [4]:
!pip install roboflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 117.2 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


**5) Download Dataset from Roboflow**

In [5]:
from roboflow import Roboflow
rf = Roboflow(api_key="ifQsBJAXMZluHn8Ipnk3")
project = rf.workspace("mohamed-traore-2ekkp").project("gtsdb---german-traffic-sign-detection-benchmark")
version = project.version(3)
dataset = version.download("yolov11")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to GTSDB---German-Traffic-Sign-Detection-Benchmark-3 in yolov11:: 100%|██████████| 2634/2634 [00:00<00:00, 9070.57it/s]


**6) Locate data.yaml**

In [6]:
DATA_YAML = f"{dataset.location}/data.yaml"
print("Dataset folder:", dataset.location)
print("data.yaml:", DATA_YAML)

Dataset folder: /content/GTSDB---German-Traffic-Sign-Detection-Benchmark-3
data.yaml: /content/GTSDB---German-Traffic-Sign-Detection-Benchmark-3/data.yaml


**7) YOLO11 Training**

In [7]:
# YOLO11 Training on Colab (German Traffic Signs)

from ultralytics import YOLO
import torch, random, numpy as np, gc

# Config
DATA_YAML = DATA_YAML        # already defined above (Roboflow export)
MODEL     = "yolo11m.pt"     # use "yolo11n.pt" if VRAM is limited
EPOCHS    = 80
IMGSZ     = 640
BATCH     = 16
RUN_NAME  = "gts_yolo11m"

# Reproducibility
def seed_everything(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
seed_everything()

# Device
device = 0 if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Train (with simple OOM fallback)
try:
    model = YOLO(MODEL)
    results = model.train(
        data=DATA_YAML,
        epochs=EPOCHS,
        imgsz=IMGSZ,
        batch=BATCH,
        device=device,
        name=RUN_NAME,
        workers=2,   # stable on Colab
        patience=20, # early stop
        cache=True   # faster, ok for portfolio
    )
except RuntimeError as e:
    if "CUDA out of memory" in str(e):
        print("OOM → retrying smaller (imgsz=512, batch=8)")
        torch.cuda.empty_cache(); gc.collect()
        model = YOLO(MODEL)
        results = model.train(
            data=DATA_YAML, epochs=EPOCHS,
            imgsz=512, batch=8, device=device,
            name=f"{RUN_NAME}_small", workers=2, patience=20
        )
    else:
        raise

# Validate
metrics = model.val(data=DATA_YAML, device=device)
print("Validation metrics:", metrics.results_dict)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Using device: 0


Ultralytics 8.3.185 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/GTSDB---German-Traffic-Sign-Detection-Benchmark-3/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=gts_yolo11m, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspecti

Overriding model.yaml nc=80 with nc=46

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytic

 13                  -1  1   1642496  ultralytics.nn.modules.block.C3k2            [1024, 512, 1, True]          
 14                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 15             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 16                  -1  1    542720  ultralytics.nn.modules.block.C3k2            [1024, 256, 1, True]          
 17                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
 18            [-1, 13]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 19                  -1  1   1511424  ultralytics.nn.modules.block.C3k2            [768, 512, 1, True]           
 20                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
 21            [-1, 10]  1         0  ultralytics.nn.modules.conv.Concat           [1]  

AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 809.0±262.1 MB/s, size: 29.5 KB)


...

                   all        108        170      0.909      0.895      0.983      0.805
          CONSTRUCTION          3          3      0.871      0.667      0.863      0.651
                DANGER          2          3          1      0.733      0.995      0.895
              NO ENTRY          2          4          1      0.867      0.995      0.782
   PEDESTRIAN CROSSING          2          2      0.984          1      0.995      0.895
       SCHOOL CROSSING          1          1      0.796          1      0.995      0.796
                  SNOW          4          7      0.975          1      0.995       0.92
                  STOP          2          2      0.866          1      0.995      0.895
                  bend          1          1      0.797          1      0.995      0.597
            bend right          2          3      0.883          1      0.995      0.811
              give way          8          9      0.965      0.889      0.902      0.746
               go lef

val: Scanning /content/GTSDB---German-Traffic-Sign-Detection-Benchmark-3/valid/labels.cache... 108 images, 6 backgrounds, 0 corrupt: 100%|██████████| 108/108 [00:00<?, ?it/s]

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



val: Caching images (0.1GB RAM): 100%|██████████| 108/108 [00:00<00:00, 308.37it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:04<00:00,  1.53it/s]


                   all        108        170      0.909      0.895      0.983      0.806
          CONSTRUCTION          3          3      0.871      0.667      0.863      0.688
                DANGER          2          3          1      0.734      0.995      0.895
              NO ENTRY          2          4          1      0.866      0.995      0.782
   PEDESTRIAN CROSSING          2          2      0.983          1      0.995      0.895
       SCHOOL CROSSING          1          1      0.797          1      0.995      0.796
                  SNOW          4          7      0.974          1      0.995      0.906
                  STOP          2          2      0.866          1      0.995      0.895
                  bend          1          1      0.798          1      0.995      0.597
            bend right          2          3      0.883          1      0.995      0.811
              give way          8          9      0.965      0.889      0.902      0.755
               go lef

In [9]:
!cp -r runs /content/drive/MyDrive/gts-artifacts
